In [ ]:
import tensorflow as tf

### clip_by_value  设置张量限幅


In [ ]:
a = tf.range(10)
a

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.maximum(a,3)      #限制下限幅

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([3, 3, 3, 3, 4, 5, 6, 7, 8, 9], dtype=int32)>

In [ ]:
tf.minimum(a,8)      #限制上限幅

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 1, 2, 3, 4, 5, 6, 7, 8, 8], dtype=int32)>

In [ ]:
tf.clip_by_value(a,3,8)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([3, 3, 3, 3, 4, 5, 6, 7, 8, 8], dtype=int32)>

### relu

In [ ]:
a = a-5
a

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([-5, -4, -3, -2, -1,  0,  1,  2,  3,  4], dtype=int32)>

In [ ]:
tf.nn.relu(a)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 0, 1, 2, 3, 4], dtype=int32)>

In [ ]:
tf.maximum(a,0)

<tf.Tensor: shape=(10,), dtype=int32, numpy=array([0, 0, 0, 0, 0, 0, 1, 2, 3, 4], dtype=int32)>

### clip_by_norm 范数限幅

In [ ]:
a = tf.random.normal([2,2],mean=10)
a

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[10.06273 ,  8.777875],
       [ 8.819092, 10.545747]], dtype=float32)>

In [ ]:
tf.norm(a)

<tf.Tensor: shape=(), dtype=float32, numpy=19.165041>

In [ ]:
#将a的范数（模长）放缩为15
aa = tf.clip_by_norm(a,15)     
aa

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[7.875848 , 6.870224 ],
       [6.9024835, 8.253893 ]], dtype=float32)>

In [ ]:
tf.norm(aa)

<tf.Tensor: shape=(), dtype=float32, numpy=14.999999>

### Gradient clipping  梯度限幅

### gradient exploding or vanishing  对于梯度爆炸或梯度消失，需要对梯度进行限幅
梯度爆炸会导致优化的时候参数来回震荡，而梯度消失会导致参数优化起来非常吃力

set lr=1   设置过小或过大都有可能发生梯度爆炸或梯度消失

new_grads, total_norm = tf.clip_by_global_norm(grads,25)

In [ ]:
import  tensorflow as tf
from    tensorflow import keras
from    tensorflow.keras import datasets, layers, optimizers
import  os

os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
print(tf.__version__)

(x, y), _ = datasets.mnist.load_data()
x = tf.convert_to_tensor(x, dtype=tf.float32) / 50.
y = tf.convert_to_tensor(y)
y = tf.one_hot(y, depth=10)
print('x:', x.shape, 'y:', y.shape)
train_db = tf.data.Dataset.from_tensor_slices((x,y)).batch(128).repeat(30)
x,y = next(iter(train_db))
print('sample:', x.shape, y.shape)
# print(x[0], y[0])



def main():

  # 784 => 512
  w1, b1 = tf.Variable(tf.random.truncated_normal([784, 512], stddev=0.1)), tf.Variable(tf.zeros([512]))
  # 512 => 256
  w2, b2 = tf.Variable(tf.random.truncated_normal([512, 256], stddev=0.1)), tf.Variable(tf.zeros([256]))
  # 256 => 10
  w3, b3 = tf.Variable(tf.random.truncated_normal([256, 10], stddev=0.1)), tf.Variable(tf.zeros([10]))



  optimizer = optimizers.SGD(lr=0.01)


  for step, (x,y) in enumerate(train_db):

    # [b, 28, 28] => [b, 784]
    x = tf.reshape(x, (-1, 784))

    with tf.GradientTape() as tape:

      # layer1.
      h1 = x @ w1 + b1
      h1 = tf.nn.relu(h1)
      # layer2
      h2 = h1 @ w2 + b2
      h2 = tf.nn.relu(h2)
      # output
      out = h2 @ w3 + b3
      # out = tf.nn.relu(out)

      # compute loss
      # [b, 10] - [b, 10]
      loss = tf.square(y-out)
      # [b, 10] => [b]
      loss = tf.reduce_mean(loss, axis=1)
      # [b] => scalar
      loss = tf.reduce_mean(loss)



    #compute gradient
    grads = tape.gradient(loss, [w1, b1, w2, b2, w3, b3])
    #print('==before==')
    #for g in grads:
    #  print(tf.norm(g))
    
    grads,  _ = tf.clip_by_global_norm(grads, 15)    #！！通过clipping操作可以非常明显的对学习过程进行优化

    #print('==after==')
    #for g in grads:
    #  print(tf.norm(g))
    #update w' = w - lr*grad
    optimizer.apply_gradients(zip(grads, [w1, b1, w2, b2, w3, b3]))



    if step % 100 == 0:
        print(step, 'loss:', float(loss))




if __name__ == '__main__':
  main()

2.4.1
x: (60000, 28, 28) y: (60000, 10)
sample: (128, 28, 28) (128, 10)
0 loss: 23.39851188659668
100 loss: 0.5639000535011292
200 loss: 0.18785670399665833
300 loss: 0.11751298606395721
400 loss: 0.08844572305679321
500 loss: 0.08472402393817902
600 loss: 0.10523843765258789
700 loss: 0.07382898032665253
800 loss: 0.07729995250701904
900 loss: 0.06435120105743408
1000 loss: 0.05755949020385742
1100 loss: 0.06252418458461761
1200 loss: 0.06925288587808609
1300 loss: 0.06465571373701096
1400 loss: 0.04530062898993492
1500 loss: 0.05128102004528046
1600 loss: 0.055705830454826355
1700 loss: 0.05883311480283737
1800 loss: 0.06530734896659851
1900 loss: 0.05067121610045433
2000 loss: 0.04421452060341835
2100 loss: 0.047242917120456696
2200 loss: 0.0543193593621254
2300 loss: 0.04162650555372238
2400 loss: 0.04668658599257469
2500 loss: 0.047709427773952484
2600 loss: 0.049346525222063065
2700 loss: 0.05850733071565628
2800 loss: 0.035876981914043427
2900 loss: 0.03817438334226608
3000 loss